# Volumetric Benchmarking of Floquet Codes

In [78]:

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from collections import defaultdict
from typing import Optional, List
import math
import sinter
import matplotlib
import rsmf
from main.codes.tic_tac_toe.gauge.GaugeFloquetColourCode import GaugeFloquetColourCode
from main.codes.tic_tac_toe.gauge.GaugeHoneycombCode import GaugeHoneycombCode
import matplotlib.patches as mpatches
#from matplotlib.ticker import MaxNLocator


formatter : plt.Figure = rsmf.setup(r"\documentclass[a4paper,11pt,noarxiv]{quantumarticle}")


# Functions used to process the data

In [79]:
def get_sinter_fit(log_ps, sqrt_qs, target_x, stat):
    if len(log_ps) < 2:
        print('error, less than 2 points')
        return None
    
    slope_fit = sinter.fit_line_slope(
        xs=log_ps,
        ys=sqrt_qs,
        max_extra_squared_error=1,
    )
    if slope_fit.best >= 0:
        return None
    
    if slope_fit.high >= 0:
        # Slope is going the wrong way! Definitely over threshold.
        print('error, slope is going the wrong way')
        return None

    fit = sinter.fit_line_y_at_x(
        xs=log_ps,
        ys=sqrt_qs,
        target_x=target_x,
        max_extra_squared_error=1,  
    )
    return (fit)


def extrapolate_footprint_achieving_error_rate(
        group: List[sinter.TaskStats],
        target_p: float,
) -> Optional[sinter.Fit]:
    """Taken from Craig Gidney's code.
    
    Args:
        group: A list of TaskStats objects.
        target_p: The target probability of failure, for teraquop use 1e-12.

    Returns:
        A Fit object representing the footprint that would achieve the target
        probability of failure, or None if the data was insufficient.
    """
    assert len({stat.json_metadata['per'] for stat in group}) == 1
    sqrt_qs = []
    log_ps = []
    for stat in group:
        if stat.shots:
            p_shot = stat.errors / stat.shots
            if 0 < p_shot < 0.5:

                p_unit = p_shot
                sqrt_qs.append(math.sqrt(stat.json_metadata['distance']**2)) # huh why am is squaring and then taking the square root?
                log_ps.append(math.log(p_unit))
            

    return get_sinter_fit(log_ps, sqrt_qs, math.log(target_p), stat)

def low_error_for_multiplication(values, max_values):
    relative_error = 0
    abs_value = 1
    for value, max_value in zip(values, max_values):
        relative_error += (abs(max_value-value)/value)**2
        abs_value *= value
    
    relative_error = math.sqrt(relative_error)
    return(abs_value*(1-relative_error))

def high_error_for_multiplication(values, max_values):
    relative_error = 0
    abs_value = 1
    for value, max_value in zip(values, max_values):
        relative_error += (abs(max_value-value)/value)**2
        abs_value *= value

    relative_error = math.sqrt(relative_error)

    return(abs_value*(1+relative_error))


# Functions used to generate plots

In [80]:

from typing import Tuple


def float_to_color_shade(value: int, color_map: matplotlib.colors.LinearSegmentedColormap, vmin=200, vmax=3000):
    """
    Convert a float between 1 and 1000 to a shade of blue.

    Parameters
    ----------
    value : float
        The float value to convert.
    vmin : float
        The minimum value of the range. Default is 1.
    vmax : float
        The maximum value of the range. Default is 1000.

    Returns
    -------
    color : tuple
        The RGBA color corresponding to the input value.
    """
    # Normalize the value to the range [0, 1]
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    normalized_value = norm(value)

    # Get the color corresponding to the normalized value
    color : Tuple[int,int,int,int]= color_map(normalized_value)
    return color

# Data processing

### Import and filter data

In [33]:
stats_memory : List[sinter.TaskStats] = sinter.stats_from_csv_files('./out/data/memory_0.001.csv')
stats_stability : List[sinter.TaskStats] = sinter.stats_from_csv_files('./out/data/stability_0.001.csv')
def filter_stats(stats, metadeta_entries: dict() = None, decoder = None):
    for metadeta_key, metadeta_value in metadeta_entries.items():
        stats = [stat for stat in stats if stat.json_metadata[metadeta_key] == metadeta_value]
    if decoder:
        stats = [stat for stat in stats if stat.decoder == decoder]
    return stats

stats_0001_memory_x_pymatching = filter_stats(stats_memory, metadeta_entries = {'per': 0.001, 'logical_observable': 'memory_x'}, decoder = 'pymatching')
stats_0001_memory_z_pymatching = filter_stats(stats_memory, metadeta_entries = {'per': 0.001, 'logical_observable': 'memory_z'}, decoder = 'pymatching')
stats_0001_memory_xz_pymatching = stats_0001_memory_x_pymatching + stats_0001_memory_z_pymatching

stats_0001_memory_x_beliefmatching = filter_stats(stats_memory, metadeta_entries = {'per': 0.001, 'logical_observable': 'memory_x'}, decoder = 'beliefmatching')
stats_0001_memory_z_beliefmatching = filter_stats(stats_memory, metadeta_entries = {'per': 0.001, 'logical_observable': 'memory_z'}, decoder = 'beliefmatching')
stats_0001_memory_xz_beliefmatching = stats_0001_memory_x_beliefmatching + stats_0001_memory_z_beliefmatching

stats_0001_stability_x_pymatching = filter_stats(stats_stability, metadeta_entries = {'per': 0.001, 'logical_observable': 'stability_x'}, decoder = 'pymatching')
stats_0001_stability_z_pymatching = filter_stats(stats_stability, metadeta_entries = {'per': 0.001, 'logical_observable': 'stability_z'}, decoder = 'pymatching')
stats_0001_stability_xz_pymatching = stats_0001_stability_x_pymatching + stats_0001_stability_z_pymatching

stats_0001_stability_x_beliefmatching = filter_stats(stats_stability, metadeta_entries = {'per': 0.001, 'logical_observable': 'stability_x'}, decoder = 'beliefmatching')
stats_0001_stability_z_beliefmatching = filter_stats(stats_stability, metadeta_entries = {'per': 0.001, 'logical_observable': 'stability_z'}, decoder = 'beliefmatching')
stats_0001_stability_xz_beliefmatching = stats_0001_stability_x_beliefmatching + stats_0001_stability_z_beliefmatching


### Calculate x-side and z-side

In [34]:
def calc_teraquop_patch_diameter(stats):
    noise_model_groups = sinter.group_by(stats, key = lambda stat: str(stat.json_metadata['px'])+ ',' + str(stat.json_metadata['py']) + ',' + str(stat.json_metadata['pz']) + ',' + str(stat.json_metadata['pm']))
    noise_model_footprints = defaultdict()

    for noise_model_key, noise_model_group in noise_model_groups.items():

        code_name_groups = sinter.group_by(noise_model_group, key = lambda stat: (stat.json_metadata['code_name'], stat.json_metadata['gf_0'], stat.json_metadata['gf_1'], stat.json_metadata['gf_2']))
        noise_model_footprints[noise_model_key] = defaultdict()
        for code_name_key, code_name_group in code_name_groups.items(): 
            noise_model_footprints[noise_model_key][code_name_key] = extrapolate_footprint_achieving_error_rate(
                    code_name_group,
                    target_p=10**(-12))
    return(noise_model_footprints)


In [35]:
x_patch_diameters = dict()
z_patch_diameters = dict()
x_patch_diameters["pymatching"] = calc_teraquop_patch_diameter(stats_0001_memory_x_pymatching)
z_patch_diameters["pymatching"] = calc_teraquop_patch_diameter(stats_0001_memory_z_pymatching)

In [36]:
x_patch_diameters["beliefmatching"] = calc_teraquop_patch_diameter(stats_0001_memory_x_beliefmatching)
z_patch_diameters["beliefmatching"] = calc_teraquop_patch_diameter(stats_0001_memory_z_beliefmatching)

### Calculate number of qubits

In [37]:
def calc_number_of_qubits(x_patch_diameters, z_patch_diamters):
    number_of_qubits = dict()
    for noise_model, diameters in x_patch_diameters.items():
        number_of_qubits[noise_model] = dict()
        for code_name, diameter in diameters.items():
            if z_patch_diamters[noise_model][code_name] and diameter:

                low_fit = 6*low_error_for_multiplication([diameter.best, z_patch_diamters[noise_model][code_name].best], [diameter.low, z_patch_diamters[noise_model][code_name].low])
                high_fit = 6*high_error_for_multiplication([diameter.best, z_patch_diamters[noise_model][code_name].best], [diameter.high, z_patch_diamters[noise_model][code_name].high])
                number_of_qubits[noise_model][code_name] = sinter.Fit(low=low_fit,
                                                                    best=6 * diameter.best * z_patch_diamters[noise_model][code_name].best,
                                                                        high=high_fit)
            else:
                print('error, no diameter', code_name, noise_model)
    return(number_of_qubits)

In [38]:
number_of_qubits = dict()
number_of_qubits["pymatching"] = calc_number_of_qubits(x_patch_diameters["pymatching"], z_patch_diameters["pymatching"])

In [39]:
number_of_qubits["beliefmatching"] = calc_number_of_qubits(x_patch_diameters["beliefmatching"], z_patch_diameters["beliefmatching"])

### Calculate hight

In [40]:
def distance_to_hight(distance_fit: sinter.Fit, code_string: str, letter):
    if code_string[0] == "GaugeFloquetColourCode":
        code = GaugeFloquetColourCode(4, [code_string[1], code_string[2]])
    elif code_string[0] == "GaugeHoneycombCode":
        code = GaugeHoneycombCode(4, [code_string[1], code_string[2], code_string[3]])
    hight_low = code.get_number_of_rounds_for_single_timelike_distance(math.ceil(distance_fit.low), letter, 'phenomenological_noise')
    hight_best = code.get_number_of_rounds_for_single_timelike_distance(math.ceil(distance_fit.best), letter,  'phenomenological_noise')
    hight_high = code.get_number_of_rounds_for_single_timelike_distance(math.ceil(distance_fit.high), letter, 'phenomenological_noise')
    return(sinter.Fit(hight_low, hight_best, hight_high))


def calc_hight(stats, letter):
    noise_model_groups = sinter.group_by(stats, key = lambda stat: str(stat.json_metadata['px'])+ ',' + str(stat.json_metadata['py']) + ',' + str(stat.json_metadata['pz']) + ',' + str(stat.json_metadata['pm']))
    noise_model_footprints = defaultdict()
    noise_model_hights = defaultdict()

    for noise_model_key, noise_model_group in noise_model_groups.items():

        code_name_groups = sinter.group_by(noise_model_group, key = lambda stat: (stat.json_metadata['code_name'], stat.json_metadata['gf_0'], stat.json_metadata['gf_1'], stat.json_metadata['gf_2']))
        noise_model_footprints[noise_model_key] = defaultdict()
        noise_model_hights[noise_model_key] = defaultdict()
        for code_name_key, code_name_group in code_name_groups.items(): 
            noise_model_footprints[noise_model_key][code_name_key] = extrapolate_footprint_achieving_error_rate(
                    code_name_group,
                    target_p=10**(-12))
            if noise_model_footprints[noise_model_key][code_name_key] != None:
                noise_model_hights[noise_model_key][code_name_key] = distance_to_hight(noise_model_footprints[noise_model_key][code_name_key], code_name_key, letter)
    return(noise_model_hights)

In [41]:
x_hights_stability=dict()
x_hights_stability["pymatching"] = calc_hight(stats_0001_stability_x_pymatching, 'X')
x_hights_stability["beliefmatching"] = calc_hight(stats_0001_stability_x_beliefmatching, 'X')


error, slope is going the wrong way


In [42]:
z_hights_stability=dict()

z_hights_stability["pymatching"] = calc_hight(stats_0001_stability_z_pymatching, 'Z')
z_hights_stability["beliefmatching"] = calc_hight(stats_0001_stability_z_beliefmatching, 'Z')


error, less than 2 points
error, less than 2 points


In [43]:
def calc_xz_hight(x_hights_stability, z_hights_stability):
    xz_hight = dict()
    for noise_model, x_hights_stability in x_hights_stability.items():
        xz_hight[noise_model] = dict()
        for code_name, x_hight in x_hights_stability.items():

            if code_name in z_hights_stability[noise_model]:
                z_hight = z_hights_stability[noise_model][code_name]
                xz_hight[noise_model][code_name] = sinter.Fit((x_hight.low + z_hight.low)/2, (x_hight.best + z_hight.best)/2, (x_hight.high + z_hight.high)/2)
            else:
                z_hight = None
                print('error, no hight', code_name, noise_model)

    return(xz_hight)



In [44]:
xz_hight = dict()
xz_hight["pymatching"] = calc_xz_hight(x_hights_stability["pymatching"], z_hights_stability["pymatching"])


In [45]:
xz_hight["beliefmatching"] = calc_xz_hight(x_hights_stability["beliefmatching"], z_hights_stability["beliefmatching"])

error, no hight ('GaugeFloquetColourCode', 1, 1, 0) 1.0,1.0,16.0,1.0
error, no hight ('GaugeFloquetColourCode', 2, 1, 0) 1.0,1.0,16.0,1.0


In [46]:
def calc_volumes(number_of_qubits, xz_hight):
    volumes = dict()
    for noise_model, code_name in number_of_qubits.items():
        volumes[noise_model] = dict()
        for code_name, number_of_qubit in code_name.items():
            if code_name in xz_hight[noise_model] and xz_hight[noise_model][code_name] != None:
                low_fit = low_error_for_multiplication([number_of_qubit.best, xz_hight[noise_model][code_name].best], [number_of_qubit.low, xz_hight[noise_model][code_name].low])
                high_fit = high_error_for_multiplication([number_of_qubit.best, xz_hight[noise_model][code_name].best], [number_of_qubit.high, xz_hight[noise_model][code_name].high])
                volumes[noise_model][code_name] = sinter.Fit(low=low_fit,
                        best=number_of_qubit.best * xz_hight[noise_model][code_name].best,
                        high=high_fit)

    return(volumes)


In [47]:
volumes = dict()
volumes["pymatching"] = calc_volumes(number_of_qubits["pymatching"], xz_hight["pymatching"])
volumes["beliefmatching"] = calc_volumes(number_of_qubits["beliefmatching"], xz_hight["beliefmatching"])

### Find best footprints

In [48]:
def get_best_volume(volumes_at_noise_model):
    best_volume_val = None
    best_volume_name = None
    for code_name, volumes in volumes_at_noise_model.items():
        if volumes.best is not None and (best_volume_val is None or volumes.best < best_volume_val):
            best_volume_val = volumes.best
            best_volume_name = code_name
    return best_volume_name, best_volume_val

def get_best_volumes(volumes):
    best_codes = dict()
    for noise_model in volumes.keys():
        best_codes[noise_model] = get_best_volume(volumes[noise_model])
    return(best_codes)

In [49]:
best_codes = dict()
best_codes['pymatching'] = get_best_volumes(volumes["pymatching"])
best_codes['beliefmatching'] = get_best_volumes(volumes["beliefmatching"])

### Functions for generating plots


In [50]:
def plot_best_footprints(best_codes: dict, ax: plt.Axes, vmin, vmax):

    x_y_to_color = dict()
    x_values = list()
    y_values = list()
    for noise_model, best_code in best_codes.items():
        x_val = float(noise_model.split(',')[-2])
        y_val = float(noise_model.split(',')[-1])
    
        if best_code[0][0] == 'GaugeHoneycombCode':
            x_y_to_color[(x_val, y_val)] = float_to_color_shade(best_code[1], plt.cm.Greens,vmin,vmax)
        elif best_code[0][0] == 'GaugeFloquetColourCode':
            x_y_to_color[(x_val, y_val)] = float_to_color_shade(best_code[1], plt.cm.Reds,vmin,vmax)    

        if x_val not in x_values:
            x_values.append(x_val)
        if y_val not in y_values:
            y_values.append(y_val)


    x_values.sort()
    y_values.sort()

    color_matrix = [[0 for i in range(len(y_values))] for j in range(len(x_values))]
    for x_val in x_values:
        for y_val in y_values:
            if (x_val, y_val) in x_y_to_color:
                color_matrix[y_values.index(y_val)][x_values.index(x_val)] = x_y_to_color[(x_val, y_val)]

    ax.imshow(color_matrix, origin = 'lower')
    ax.set_xticks(np.arange(len(x_values)), labels=x_values)
    ax.set_yticks(np.arange(len(y_values)), labels=y_values)
    ax.set_xlabel('Z error bias')
    ax.set_ylabel('Measurement error bias',)
    ax.plot([0.0], [0.0], marker='$\\clubsuit$', markersize=10, markeredgewidth=0, color='black')
    ax.plot([3], [3], marker='$\\diamondsuit$', markersize=10,markeredgewidth=0, color='black')
    ax.plot([0], [3], marker='$\\heartsuit$', markersize=10,markeredgewidth=0, color='black')  # Added square marker
    ax.plot([3], [0], marker='$\\spadesuit$', markersize=10, markeredgewidth=0, color='black')

def format_code_label(code_name):
    code_name_to_label = {
        'GaugeHoneycombCode': 'HCC',
        'GaugeFloquetColourCode': 'FCC'
    }
    if code_name_to_label.get(code_name[0], code_name) == 'HCC':
        return(f"$X^{code_name[1:][0]}Y^{code_name[1:][1]}Z^{code_name[1:][2]}$")
    else:
        return(f"$X^{code_name[1:][0]}Z^{code_name[1:][1]}$")



def plot_footprints(footprints,error_model, ax=plt, top_n=10, vmin=10,vmax=80, y_label = 'Patch diameter', marker=None, ymax=None):

    sorted_items = sorted(footprints[error_model].items(), key=lambda item: item[1].best if item[1] is not None else float('inf'))
    code_name_to_color = {'GaugeFloquetColourCode': plt.cm.Reds, 'GaugeHoneycombCode': plt.cm.Greens}
    for code_name, footprint in sorted_items[:top_n]:
        formatted_label = format_code_label(code_name)            
        bars = ax.bar(str(code_name), float(footprint.best), color = float_to_color_shade(footprint.best, code_name_to_color.get(code_name[0], 'black'), vmin=vmin, vmax=vmax))
        ax.errorbar(str(code_name), float(footprint.best), yerr=[[float(footprint.best - footprint.low)], [float(footprint.high - footprint.best)],], fmt='o', color='black', ecolor='black', elinewidth=1, capsize=3)
        for bar in bars:
            height = 1.03*footprint.high
            ax.text(bar.get_x() + bar.get_width() / 2.0, height, formatted_label, ha='center', va='bottom', rotation=90)
    ax.set_ylim(0, footprint.high + footprint.best)

    if y_label is not None:
        ax.set_ylabel(y_label)
    ax.set_xticks([]) 
    ax.set_ylim(0, ymax)
    if marker is not None:

        ax.plot([0], [0.9*ymax], marker=marker, markersize=10, markeredgewidth = 0, color='black')
 
 
def draw_corner_bar_plots(footprints, fig, axd, n_points, vmin, vmax, ylabel, ymax):
    plot_footprints(footprints, '1.0,1.0,1.0,16.0', axd['A'],n_points, vmin=vmin, vmax=vmax, y_label=ylabel,marker='$\\heartsuit$', ymax=ymax) 
    plot_footprints(footprints, '1.0,1.0,16.0,16.0', axd['C'],n_points, vmin=vmin, vmax=vmax, y_label=None, marker='$\\diamondsuit$', ymax=ymax)

    plot_footprints(footprints, '1.0,1.0,1.0,1.0', axd['D'],n_points, vmin=vmin, vmax=vmax, y_label=ylabel, marker='$\\clubsuit$', ymax=ymax) 
    plot_footprints(footprints, '1.0,1.0,16.0,1.0', axd['E'], n_points, vmin=vmin, vmax=vmax, y_label=None, marker='$\\spadesuit$' ,ymax=ymax)
    fig.tight_layout()


# Generate plots

In [87]:
def create_overview_plot(decoder: str,vmin: int, vmax: int, ymax: int):

    layout = [['B',   'B'],
            [ 'B',   'B',],
            ['title1', 'title1'],
            [ 'A', 'C'],
            ['D', 'E']]
    fig : plt.Figure = formatter.figure(width_ratio=2, aspect_ratio=1.5)
    axd = fig.subplot_mosaic(layout, height_ratios=[0.5, 0.5, 0.2, 1, 1],)
    fig.tight_layout()

    axd['B'].set_title('Volumes of the best code at different biases')
    axd['title1'].set_title(r'Volumes of the 10 best codes at biases $\heartsuit, \diamondsuit,\clubsuit, \spadesuit $', y=-0.1)
    axd['title1'].axis('off')


    plot_best_footprints(best_codes[decoder], axd['B'], vmin, vmax)
    draw_corner_bar_plots(volumes[decoder], fig, axd, 10, vmin, vmax, r"Teraquop volume", ymax)


    cb1 = plt.colorbar(mappable=plt.cm.ScalarMappable(norm=mcolors.Normalize(vmin=vmin/1e6, vmax=vmax/1e6), cmap=plt.cm.Greens), ax=axd['B'], label=r'Volume $\times 10^6$ of $X^a Y^b Z^c$  DCCC ')
    cb2 = plt.colorbar(mappable=plt.cm.ScalarMappable(norm=mcolors.Normalize(vmin=vmin/1e6, vmax=vmax/1e6), cmap=plt.cm.Reds), ax=axd['B'], label=r'Volume $\times 10^6$ of $X^a Z^b$ DCCC')
    fig.tight_layout()
    l, b, w, h = axd['B'].get_position().bounds
    axd['B'].set_position([l, b+0.05, w, h])
    l, b, w, h = cb1.ax.get_position().bounds
    cb1.ax.set_position([l, b+0.05, w, h])
    l, b, w, h = cb2.ax.get_position().bounds
    cb2.ax.set_position([l, b+0.05, w, h])
    plt.tight_layout()
    fig.savefig(f'plots/spacetime_volume_heatmap_{decoder}.pdf', bbox_inches='tight')
    plt.close(fig)


# Figure 1

In [88]:
create_overview_plot('pymatching', 2e5, 3e6, 7e6)

# Figure 2

In [89]:
create_overview_plot('beliefmatching', 2e5, 3e6, 4e6)

In [54]:
def filter_teraquop_values(teraquop_value, error_model, decoder, codes = [('GaugeFloquetColourCode', 1, 1, 0), ('GaugeHoneycombCode', 1, 1, 1)]):
    teraquop_values_all_codes = [diameter for key,diameter in teraquop_value[decoder].items() if key == error_model]
    filtered_teraquop_value = {key: diameter for key,diameter in teraquop_values_all_codes[0].items() if key in codes}
    return filtered_teraquop_value

def plot_bar(pos, value: sinter.Fit, ax: plt.Axes, code_name: str, color: str, hatch: str):
    bars = ax.bar(pos, float(value.best), color=color, hatch=hatch, edgecolor='black')
    ax.errorbar(pos, float(value.best), yerr=[[float(value.best - value.low)], [float(value.high - value.best)],], fmt='o', color='black', ecolor='black', elinewidth=1, capsize=3)

    return bars

def plot_teraquop_values(all_teraquop_values: List, error_model: str, codes: List, ax=plt, x_labels=[], vmin=10,vmax=80, ylim = None, colors=['Red', 'Green'], ylabel=None):
    pos = 0

    code_name_to_color = {code: color for code, color in zip(codes, colors)}

    for teraquop_values in all_teraquop_values:
        teraquop_values_pymatching = filter_teraquop_values(teraquop_values, error_model, 'pymatching',codes)
        teraquop_values_beliefmatching = filter_teraquop_values(teraquop_values, error_model, 'beliefmatching',codes)

        for code_name, teraquop_value_pm in teraquop_values_pymatching.items():
            plot_bar(pos, teraquop_value_pm, ax, code_name, code_name_to_color.get(code_name, 'black'), None )
            if teraquop_value_pm.best > ylim:
                ax.text(0.99*pos, 1.01*ylim, f'{teraquop_value_pm.best:.1e}'.replace('e+0', 'e'), ha='center', va='bottom', fontsize='xx-small')
            pos += 1
            teraquop_value_bm = teraquop_values_beliefmatching.get(code_name)
            plot_bar(pos, teraquop_value_bm, ax, code_name, code_name_to_color.get(code_name, 'black'), 'x')
            if teraquop_value_bm.best > ylim:
                ax.text(1.01*pos, 1.01*ylim, f'{teraquop_value_bm.best:.1e}'.replace('e+0', 'e'), ha='center', va='bottom', fontsize='xx-small')
            if ylabel is not None:
                ax.set_ylabel(ylabel)
            pos += 1.2
        pos += 1
    

    formatter = matplotlib.ticker.ScalarFormatter(useMathText=True)
    formatter.fontsize = 11
    formatter.set_scientific(True)
    formatter.set_powerlimits((-1, 1))
    ax.yaxis.set_major_formatter(formatter)
    ax.set_xticks(x_labels) 
    ax.set_ylim(0, ylim)

# Figure 17

In [55]:
fig = formatter.figure(wide=True)
subfigs = fig.subfigures(nrows=3, ncols=1)
codes = [('GaugeFloquetColourCode', 1, 1, 0), ('GaugeHoneycombCode', 1, 1, 1)]

for error_model, title, subfig in zip(['1.0,1.0,1.0,1.0', '1.0,1.0,8.0,1.0', '1.0,1.0,1.0,8.0'], ['Z error bias = 1, Measurement bias = 1', 'Z error bias = 8, Measurement bias = 1', 'Z error bias = 1, Measurement bias = 8'], subfigs):
    axs = subfig.subplots(nrows=1, ncols=2, gridspec_kw={'width_ratios': [3, 1]})
    plot_teraquop_values([x_patch_diameters, z_patch_diameters, x_hights_stability, z_hights_stability],error_model,codes, axs[0], ylim=3e2)
    plot_teraquop_values([volumes], error_model, codes, axs[1], ylim=3e6, ylabel='Teraquop volume')
    subfig.suptitle(title, y=1.05, x=0.45, fontsize='medium')

red_patch_pm = mpatches.Patch(label='$X^1Z^1$ MWPM', edgecolor='black', hatch='//', facecolor='red')
red_patch_bm = mpatches.Patch(label='$X^1Z^1$ beliefmatching', edgecolor='black', facecolor='red')
green_patch_pm = mpatches.Patch(facecolor='green', label='$X^1Y^1Z^1$ MWPM', edgecolor='black', hatch='//')
green_patch_bm = mpatches.Patch(facecolor='green', label='$X^1Y^1Z^1$ beliefmatching', edgecolor='black')

x_labels = ['$\\tilde{n}_E$', '$\\tilde{n}_M$', '$h_E$', '$h_M$']
axs[0].set_xticks([1.6,7,12.375,17.8], x_labels)
axs[0].legend(handles=[red_patch_pm, red_patch_bm, green_patch_pm, green_patch_bm], loc='upper center', bbox_to_anchor=(0.75, -0.15), ncol=2, fontsize='small')

fig.savefig('plots/X1Y1Z1_X1Z1_comparison.pdf', bbox_inches='tight')
plt.close(fig)  

# Figure 25

In [84]:
fig = formatter.figure(wide=True)
subfigs = fig.subfigures(nrows=3, ncols=1)
codes = [('GaugeFloquetColourCode', 1, 1, 0), ('GaugeFloquetColourCode', 3, 3, 0)]

for error_model, title, subfig in zip(['1.0,1.0,1.0,1.0', '1.0,1.0,8.0,1.0', '1.0,1.0,1.0,8.0'], ['Z error bias = 1, Measurement bias = 1', 'Z error bias = 8, Measurement bias = 1', 'Z error bias = 1, Measurement bias = 8'], subfigs):
    axs = subfig.subplots(nrows=1, ncols=2, gridspec_kw={'width_ratios': [3, 1]})
    plot_teraquop_values([x_patch_diameters, z_patch_diameters, x_hights_stability, z_hights_stability],error_model,codes, axs[0], ylim=3e2, colors=['red', '#4477aa'])
    plot_teraquop_values([volumes], error_model, codes, axs[1], ylim=3e6, colors=['red', '#4477aa'], ylabel='Teraquop volume')
    subfig.suptitle(title, y=1.05, x=0.45, fontsize='medium')

red_patch_pm = mpatches.Patch(label='$X^1Z^1$ MWPM', edgecolor='black', hatch='//', facecolor='red')
red_patch_bm = mpatches.Patch(label='$X^1Z^1$ beliefmatching', edgecolor='black', facecolor='red')
green_patch_pm = mpatches.Patch(facecolor='#4477aa', label='$X^3Z^3$ MWPM', edgecolor='black', hatch='//')
green_patch_bm = mpatches.Patch(facecolor='#4477aa', label='$X^3Z^3$ beliefmatching', edgecolor='black')

x_labels = ['$\\tilde{n}_E$', '$\\tilde{n}_M$', '$h_E$', '$h_M$']
axs[0].set_xticks([1.6,7,12.375,17.8], x_labels)
axs[0].legend(handles=[red_patch_pm, red_patch_bm, green_patch_pm, green_patch_bm], loc='upper center', bbox_to_anchor=(0.75, -0.15), ncol=2, fontsize='small')

fig.savefig('plots/X1Z1_X3Z3_comparison.pdf', bbox_inches='tight')
plt.close(fig)  

# Figure 24

In [57]:
fig = formatter.figure(wide=True)
error_model = '1.0,1.0,1.0,16.0'


codes = [('GaugeHoneycombCode', 1, 1, 1), ('GaugeHoneycombCode', 2, 1, 1),
         ('GaugeHoneycombCode', 2, 2, 1), ('GaugeHoneycombCode', 2, 2, 2),
         ('GaugeHoneycombCode', 3, 2, 2), ('GaugeHoneycombCode', 3, 3, 2),('GaugeHoneycombCode', 3, 3, 3) ]

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

ax = fig.add_subplot(111)
title = 'Z error bias = 1, Measurement bias = 1'
plot_teraquop_values([volumes], error_model, codes, ax, ylim=1e7, colors=colors, ylabel='Teraquop volume')
handles = []
handles.append(mpatches.Patch(label='MWPM',edgecolor='black', facecolor='white'))
handles.append(mpatches.Patch(label='beliefmatching',edgecolor='black', facecolor='white', hatch='//'))
for i, (code, color) in enumerate(zip(codes, colors)):
    code_label = f'$X^{code[1]}Y^{code[2]}Z^{code[3]}$'
    pm_patch = mpatches.Patch(label=f'{code_label}', 
                             edgecolor='black', 
                             facecolor=color)
    handles.append(pm_patch)



ax.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.5, -0.15), 
         ncol=5, columnspacing=1, handletextpad=0.5, fontsize='small')

ax.set_title('Z error bias = 1, measurement bias = 16')
ax.set_ylabel('Teraquop volume')
fig.tight_layout()
fig.savefig('plots/X1Y1Z1_X2Y2Z2_X3Y3Z3_comparison.pdf', bbox_inches='tight')
plt.close(fig)  